In [21]:

import networkx as nx
import matplotlib.pyplot as plt

LAMBDA = 1e-4 # Random number for now

class Component:
    def __init__(self, name, residence_time):
        self.name = name
        self.residence_time = residence_time
        self.input_ports = {}  # Dictionary where the key is the port name and the value is the port object
        self.output_ports = {}  # Dictionary where the key is the port name and the value is the port object
        self.tritium_inventory = 0

    def add_input_port(self, port_name, incoming_fraction=1.0):
        if not (0 <= incoming_fraction <= 1):
            raise ValueError("Incoming fraction must be between 0 and 1")
        port = Port(port_name)
        port.incoming_fraction = incoming_fraction
        self.input_ports[port_name] = port
        return port

    def add_output_port(self, port_name):
        port = Port(port_name)
        self.output_ports[port_name] = port
        return port

    def __str__(self):
        return f"{self.name}: Residence Time = {self.residence_time}, Tritium Inventory = {self.tritium_inventory}"

    def add_tritium(self, amount):
        self.tritium_inventory += amount

    def remove_tritium(self, amount):
        if self.tritium_inventory >= amount:
            self.tritium_inventory -= amount
            return amount
        else:
            removed_amount = self.tritium_inventory
            self.tritium_inventory = 0
            return removed_amount
        
    def get_inflow(self):
        inflow = 0
        for port in self.input_ports.values():
            inflow += port.flow_rate
        return inflow   
        
    def get_outflow(self):
        return self.tritium_inventory / self.residence_time
        
    def calculate_inventory_derivative(self):
        inflow = self.get_inflow()
        outflow = self.get_outflow()
        decay = self.tritium_inventory * LAMBDA  # assuming decay_rate is a constant
        return inflow - outflow - decay


class Port:
    def __init__(self, name, incoming_fraction=1.0):
        self.name = name
        self.flow_rate = 0
        self.incoming_fraction = incoming_fraction
        
    def set_flow_rate(self, flow_rate):
        self.flow_rate = flow_rate
    
class ComponentMap:
    def __init__(self):
        self.components = {}
        self.connections = {}

    def add_component(self, component):
        self.components[component.name] = component

    def connect_ports(self, component1, port1, component2, port2):
        if component1.name not in self.connections:
            self.connections[component1.name] = {}
        if component2.name not in self.connections:
            self.connections[component2.name] = {}

        self.connections[component1.name][port1.name] = (component2.name, port2.name)
        self.connections[component2.name][port2.name] = (component1.name, port1.name)
        if port1 and port2:
                port1.set_flow_rate(component1.get_outflow())
                port2.set_flow_rate(component1.get_outflow() * port2.incoming_fraction)

    def disconnect_ports(self, component1, port1, component2, port2):
        if component1.name in self.connections and port1.name in self.connections[component1.name]:
            del self.connections[component1.name][port1.name]
        if component2.name in self.connections and port2.name in self.connections[component2.name]:
            del self.connections[component2.name][port2.name]

    def get_connected_ports(self, component, port):
        if component.name in self.connections and port.name in self.connections[component.name]:
            connected_component_name, connected_port_name = self.connections[component.name][port.name]
            connected_component = self.components[connected_component_name]
            connected_port = connected_component.input_ports[connected_port_name]
            return connected_component, connected_port
        else:
            return None, None
        
    def print_connected_map(self):
        for component_name, ports in self.connections.items():
            print(f"Component: {component_name}")
            for port_name, (connected_component_name, connected_port_name) in ports.items():
                print(f"  Port: {port_name} -> Connected Component: {connected_component_name}, Connected Port: {connected_port_name}")


In [22]:
def visualize_connections(component_map):
    # Create a directed graph
    G = nx.DiGraph()

    # Add nodes for each component
    for component in component_map.components.values():
        G.add_node(component.name)

    # Add edges for each connection
    for component_name, ports in component_map.connections.items():
        for port_name, (connected_component_name, _) in ports.items():
            G.add_edge(component_name, connected_component_name)

    # Draw the graph
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10, edge_color='gray', arrows=True)

    # Show the plot
    plt.show()


In [23]:
component1 = Component("Component 1", 3)
component2 = Component("Component 2", 1)
component1.tritium_inventory = 10
port1 = component1.add_output_port("Port 1")
port2 = component2.add_input_port("Port 2", 0.5)

component_map = ComponentMap()
component_map.add_component(component1)
component_map.add_component(component2)
component_map.connect_ports(component1, port1, component2, port2)
component_map.print_connected_map()

Component: Component 1
  Port: Port 1 -> Connected Component: Component 2, Connected Port: Port 2
Component: Component 2
  Port: Port 2 -> Connected Component: Component 1, Connected Port: Port 1


In [6]:
# Create the components
component1 = Component("Component 1", 5)
component2 = Component("Component 2", 3)
component3 = Component("Component 3", 4)
# component4 = Component("Component 4", 6)
# component5 = Component("Component 5", 2)

# # Create the ports
port1 = component1.add_output_port("Port 1")
port2 = component2.add_input_port("Port 2")
port3 = component2.add_output_port("Port 3")
port4 = component3.add_input_port("Port 4")
# port5 = component3.add_output_port("Port 5")
# port6 = component4.add_input_port("Port 6")
# port7 = component4.add_output_port("Port 7")
# port8 = component5.add_input_port("Port 8")
# port9 = component5.add_output_port("Port 9")  # Additional port between Component 3 and Component 5
# port10 = component5.add_output_port("Port 10")  # Additional port for Component 5
# port11 = component1.add_input_port("Port 11")  # Additional port for Component 1

# # Connect the ports
# component_map = ComponentMap()
# component_map.connect_ports(component1, port1, component2, port2)
# component_map.connect_ports(component2, port3, component3, port4)
# component_map.connect_ports(component3, port5, component4, port6)
# component_map.connect_ports(component4, port7, component5, port8)
# component_map.connect_ports(component5, port9, component3, port5)  # Connect the additional port
# component_map.connect_ports(component5, port10, component1, port11)  # Connect the additional port
# Example usage:
# ... add components and connections to the component_map ...
# visualize_connections(component_map)



In [10]:

# Create an instance of the ComponentMap class
component_map = ComponentMap()

# Add the components to the component map
component_map.add_component(component1)
component_map.add_component(component2)
component_map.add_component(component3)

# Connect the ports
component_map.connect_ports(component1, port1, component2, port2)
component_map.connect_ports(component2, port3, component3, port4)

# Add tritium to one of the components
component1.add_tritium(10)

# Print the initial state of the components
print(component1)
print(component2)
print(component3)

# Calculate and print the initial tritium flow between the components
print("Initial Tritium Flow:", component1.get_outflow())

# Remove tritium from one of the components
component1.remove_tritium(5)

# Print the updated state of the components
print(component1)
print(component2)
print(component3)

# Calculate and print the updated tritium flow between the components
print("Updated Tritium Flow:", component1.get_outflow())

Component 1: Residence Time = 5, Tritium Inventory = 35
Component 2: Residence Time = 3, Tritium Inventory = 0
Component 3: Residence Time = 4, Tritium Inventory = 0
Initial Tritium Flow: 7.0
Component 1: Residence Time = 5, Tritium Inventory = 30
Component 2: Residence Time = 3, Tritium Inventory = 0
Component 3: Residence Time = 4, Tritium Inventory = 0
Updated Tritium Flow: 6.0
